In [34]:
from __future__ import print_function

import cobra
import cobra.test
import os
import libsbml
import pandas as pd

In [92]:
files = [x for x in list(os.listdir()) if 'sbml' in x and 'GMM' not in x]
files
hm01, hm03, hm06, hm07, hm14, hm17, hm43, hm54, hm56, hm57, hm66, hm68, hm86 = map(cobra.io.read_sbml_model, files)
models = [hm01, hm03, hm06, hm07, hm14, hm17, hm43, hm54, hm56, hm57, hm66, hm68, hm86]

for mod in models:
    print(mod)
    print(mod.optimize().objective_value)


A__HM01_model
367.09037869609216
A__HM03_model
368.9659344994808
A__HM06_model
380.3053646824794
A__HM07_model
368.96593391685104
A__HM14_model
368.96593633059655
A__HM17_model
380.3053614225452
A__HM43_model
380.3053620230616
A__HM54_model
380.3053627951524
A__HM56_model
379.87804614097877
A__HM57_model
380.4450749899052
A__HM66_model
380.3053607362397
A__HM68_model
357.73198838712284
A__HM86_model
380.30536219463795


In [9]:
#Load in gmm models, assume gmm stands for glucose minimal media
gmm_files = [x for x in list(os.listdir()) if 'GMM' in x]
hm06_gmm, hm07_gmm, hm57_gmm = map(cobra.io.read_sbml_model, gmm_files)
for mod in [hm06_gmm, hm07_gmm, hm57_gmm]:
    print(mod)
    print(mod.optimize().objective_value)
    
#either gmm stands for something else, or there isn't much difference. Or these small
#differnces actually matter a lot
#or need to optimize what I'm optimizing


A__HM06_GMM_model
368.74357080384647
A__HM07_GMM_model
369.0287958645253
A__HM57_GMM_model
368.8789568216552


In [16]:
# Just checking that biomass reaction has been set as the objective

from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(hm01)
hm01.objective = hm01.reactions.biomass0

In [46]:
#aerobic
#looking at 3 strains Arwen chose
aerobic = pd.DataFrame()
strains = [hm06, hm07, hm57]
iron_influx = [-1000, -100, -10, -1, -0.1, 0,]
aerobic['iron_influx'] = iron_influx
#iron = strain.reactions.EX_cpd10515_e0
for strain in strains:
    iron = strain.reactions.EX_cpd10515_e0
    biomass = []    
    for i in iron_influx:
        iron.lower_bound = i
        biomass.append(round(strain.optimize().objective_value, 4))
    aerobic[strain] = biomass
aerobic


,iron_influx,A__HM06_model,A__HM07_model,A__HM57_model
0,-1000.0,380.3054,368.9659,380.4451
1,-100.0,380.3054,368.9659,380.4451
2,-10.0,380.3054,368.9659,380.4451
3,-1.0,322.9487,322.9487,322.9487
4,-0.1,32.2949,32.2949,32.2949
5,0.0,0.0000,0.0000,0.0000


In [79]:
#anaerobic
anaerobic = pd.DataFrame()
anaerobic['iron_influx'] = iron_influx
for strain in strains:
    ox = strain.reactions.EX_cpd00007_e0
    ox.lower_bound = 0
    iron = strain.reactions.EX_cpd10515_e0
    biomass = []    
    for i in iron_influx:
        iron.lower_bound = i
        biomass.append(round(strain.optimize().objective_value, 4))
    anaerobic[strain] = biomass
anaerobic

#Clearly not good models - can't even differentiate between aerobic and anaerobic!!!!


,iron_influx,A__HM06_model,A__HM07_model,A__HM57_model
0,-1000.0,380.3054,368.9659,380.4451
1,-100.0,380.3054,368.9659,380.4451
2,-10.0,380.3054,368.9659,380.4451
3,-1.0,322.9487,322.9487,322.9487
4,-0.1,32.2949,32.2949,32.2949
5,0.0,0.0000,0.0000,0.0000


In [53]:
hm06.reactions.EX_cpd00007_e0.lower_bound = 0
hm06.reactions.EX_cpd10515_e0.lower_bound = -1000
hm06.optimize().objective_value

380.3053646824809

In [62]:
for met in hm01.metabolites:
    if 'O2' in met.name:
        print(met, ' name ', met.name)
        

cpd00011_c0  name  CO2_c0
cpd00007_c0  name  O2_c0
cpd00025_c0  name  H2O2_c0
cpd00532_c0  name  O2__c0
cpd03736_c0  name  Lauroyl_KDO2_lipid_IVA_c0
cpd00007_e0  name  O2_e0
cpd00011_e0  name  CO2_e0


In [72]:
ox = hm01.reactions.EX_cpd00007_e0
ox.lower_bound = -1000
hm01.optimize().objective_value
hm01.summary()
#no in flux for oxygen :S

IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  -------------
cpd00024_e0    1e+03  cpd00009_e0    1e+03  biomass0  161
cpd00080_e0    1e+03  cpd00011_e0    1e+03
cpd00132_e0    1e+03  cpd00013_e0    1e+03
cpd00367_e0    1e+03  cpd00033_e0    1e+03
cpd03279_e0  946      cpd00036_e0    1e+03
cpd00794_e0  752      cpd00092_e0    1e+03
cpd11589_e0  732      cpd00106_e0    1e+03
cpd00054_e0  540      cpd00396_e0    1e+03
cpd00079_e0  500      cpd00108_e0  984
cpd00314_e0  436      cpd00122_e0  984
cpd17041_c0  161      cpd00100_e0  963
cpd17042_c0  161      cpd00309_e0  919
cpd17043_c0  161      cpd00129_e0  536
cpd00137_e0  104      cpd01015_e0  439
cpd00051_e0   88.7    cpd00047_e0  359
cpd00653_e0   87.9    cpd00065_e0  338
cpd00156_e0   70.6    cpd00067_e0  224
cpd15604_e0   60.6    cpd00012_e0  182
cpd00039_e0   50.1    cpd11416_c0  161
cpd11593_e0   46.3    cpd00064_e0   88.7
cpd00322_e0   39      cpd00355_e0    5.13
cpd00412_e0   33

In [80]:
hm01.reactions.rxn00117_c0

Reaction identifier,rxn00117_c0
Name,ATP_UDP_phosphotransferase_c0
Memory address,0x011c943668
Stoichiometry,cpd00002_c0 + cpd00014_c0 <=> cpd00008_c0 + cpd00062_c0 ATP_c0 + UDP_c0 <=> ADP_c0 + UTP_c0
GPR,fig|562.23995.peg.2275
Lower bound,-1000.0
Upper bound,1000.0


In [107]:
hm03.objective = "rxn00117_c0"
#hm01.reactions.get_by_id("rxn00117_c0").upper_bound = 1000.0
linear_reaction_coefficients(hm01)

hm03.reactions.EX_cpd00007_e0.lower_bound = -1000
#hm01.reactions.EX_cpd00007_e0.upper_bound = 1000
hm03.optimize()
hm03.summary()


IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  ------------------
cpd00024_e0    1e+03  cpd00009_e0    1e+03  rxn00117_c0  1e+03
cpd00080_e0    1e+03  cpd00011_e0    1e+03
cpd00132_e0    1e+03  cpd00013_e0    1e+03
cpd00276_e0    1e+03  cpd00036_e0    1e+03
cpd00367_e0    1e+03  cpd00092_e0    1e+03
cpd03279_e0    1e+03  cpd00309_e0    1e+03
cpd00249_e0  782      cpd03198_e0    1e+03
cpd00079_e0  770      cpd00122_e0  967
cpd00054_e0  758      cpd00033_e0  936
cpd00794_e0  636      cpd00035_e0  832
cpd17041_c0  334      cpd00412_e0  719
cpd17042_c0  334      cpd00082_e0  488
cpd17043_c0  334      cpd01912_e0  479
cpd00235_e0  208      cpd11416_c0  334
cpd00107_e0  154      cpd00047_e0  233
cpd00156_e0  117      cpd00138_e0  208
cpd01217_e0  104      cpd00573_e0  162
cpd00039_e0  104      cpd00012_e0  116
cpd00051_e0   82.3    cpd00130_e0   54.8
cpd00322_e0   80.7    cpd00065_e0   46.2
cpd00129_e0   61.5    cpd00139_e0    3.1
cpd11580_

## Modeling different enviromental conditions:

   * Trying to figure out how to model iron depletion. What's the id for extracellular iron? What reactions is it involved in? Can I set them to 0.



In [29]:
for met in hm01.metabolites:
    if 'Fe' in met.name:
        print(met, ' name ', met.name)

cpd10515_e0  name  Fe2_e0
cpd10515_c0  name  Fe2_c0
cpd03724_e0  name  Ferrichrome_e0
cpd03724_c0  name  Ferrichrome_c0
cpd02755_c0  name  Fecosterol_c0
cpd01733_c0  name  Ferulaldehyde_c0
cpd01059_c0  name  Ferulate_c0
cpd03725_e0  name  FeIIIdicitrate_e0
cpd03725_c0  name  FeIIIdicitrate_c0


In [31]:
hm01.metabolites.cpd10515_e0.reactions

frozenset({<Reaction rxn05555_c0 at 0x11101fa58>,
           <Reaction EX_cpd10515_e0 at 0x1116202e8>})

In [32]:
hm01.reactions.EX_cpd10515_e0

Reaction identifier,EX_cpd10515_e0
Name,EX_Fe2_e0
Memory address,0x01116202e8
Stoichiometry,cpd10515_e0 <=> Fe2_e0 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [69]:
hm07.reactions.EX_cpd10515_e0

Reaction identifier,EX_cpd10515_e0
Name,EX_Fe2_e0
Memory address,0x0113876b00
Stoichiometry,cpd10515_e0 <=> Fe2_e0 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [94]:
hm07.reactions.EX_cpd10515_e0.upper_bound = 0.001
hm07.reactions.EX_cpd10515_e0.lower_bound = -0.001
hm07.optimize().objective_value

0.32294871794871766

In [96]:
hm06.reactions.EX_cpd10515_e0.upper_bound = 0.001
hm06.reactions.EX_cpd10515_e0.lower_bound = 0
hm06.optimize().objective_value

0.0

* I think these models are too crude to see differnces between strains like that, or maybe it's my attempts that are too crude
* Might still want to come up with a list of reactions that are different between the models
* Getting worried about getting meaningful results out of these data